# Evidenztheorie für Datensatz "wohnungen_1.csv"

## Vorbereitung
Zuerst importieren wir die benötigten Bilbiotheken und Funktionen.

In [1]:
import pandas as pd
from data_cleaning.text_to_numeric import get_zimmer, get_stockwerk, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_stadtmitte, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter
from combined_mass.combined_mass import calculate_combined_mass

## Reinigung des Datensatzes

Um das Datensatz korrekt benutzen zu können, müssen wir den fließtext durch Zahlen ersetzen damit die KI mit diesen Daten arbeiten kann. Dafür laden wir zuerst die CSV-Datei, dann entfernen wir die Zeilen wo keine Bewohnerklasse ist und verwenden anschließend die Funktionen aus der `data_cleaning.py` um den ganzen Fließtext der Datei in Zahlen umzuwandeln.

In [2]:
file_path = '../../data/Wohnungen_2.csv'
data = pd.read_csv(file_path, sep=';')

bewohnerkategorie = ["Studierende", "Kleinfamilie", "DINK", "SingleHighIncome", "Expatriate", "Rentnerpaar"]
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(lambda row: 'Keine' if all(row == 'nein') else row.idxmax(), axis=1)

data = data[data['Bewohnerkategorie'] != 'Keine']

columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Entfernung zur Stadtmitte'] = data['Entfernung zur Stadtmitte'].apply(get_stadtmitte)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

    Zimmerzahl  Stockwerk  Hausmeister  Kindergarten  Schule  S-Bahn  Garage  \
3            0          3            1             2       0       0       0   
4            2          4            0             1       2       0       0   
6            2          1            0             2       2       3       0   
7            2          0            0             2       2       3       1   
16           0          4            0             0       2       2       1   

    Miete  Nebenkosten  Alter  ...  Kehrwoche  Moebliert  Quadratmeter  \
3     276           50     63  ...          0          0            25   
4     626          226     18  ...          0          0            96   
6     626          226      6  ...          0          0            96   
7     726          226      0  ...          0          0           106   
16    276           75     26  ...          1          0            25   

    Studierende  Kleinfamilie  DINK  SingleHighIncome  Expatriate  \
3    

/tmp/ipykernel_3681/2299712992.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


## Anwendung der künstlichen Intelligenz

Jetzt bestimmen wir die Menge Omega mit den Attributen die zu bestimmen sind, also die Bewohnerkategorie.

In [3]:
omega = ["Studierende", "Kleinfamilie", "DINK", "SingleHighIncome", "Expatriate", "Rentnerpaar"]

Jetzt können wir ein Beispiel angeben mit Attributen. WIe im Beispiel darunter wo man schauen kann mit `'Schule': get_schule("nah")`. Die kombinierte Evidenz wird dann durch die Funktion `calculate_combined_mass()` berechnet die diese durch unsere gereinigten Daten, den gegebenen Attributen und des Omegas berechnet wird. Am Ende wird durch die Methode `max_bel()` das Attribut aus Omega wiedergegeben die die Maximalste Glaubwürdigkeit von allen hat.

In [4]:
# Gegebene Attribut-Wert-Kombinationen\n",
attributes_values = {
    'Zimmerzahl': 0,
    'Stockwerk': 3,
    'Schule': get_schule("nah"),
}

# Berechne die kombinierte Evidenz
combined_mass = calculate_combined_mass(data, attributes_values, omega)

# Ausgabe der kombinierten Evidenz
print("Kombinierte Evidenz:", combined_mass)

# Ausgabe der Glaubwürdigkeit (Belief) für alle Hypothesen
print("\nGlaubwürdigkeit für alle Hypothesen:")
for hypothesis in combined_mass.get_hypotheses():
    belief = combined_mass.bel(hypothesis)
    print(f"Hypothese: {hypothesis}, Glaubwürdigkeit: {belief:.4f}")

# Wahrscheinlichste Hypothese basierend auf der kombinierten Evidenz
most_likely = combined_mass.max_bel()  # Maximale Glaubwürdigkeit

print("Am wahrscheinlichsten ist:", most_likely)

Basismaße für Zimmerzahl=0: {set():0.7837837837837838; {'Kleinfamilie'}:0.21621621621621623; {'Studierende'}:0.0; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Stockwerk=3: {set():0.8378378378378378; {'Studierende'}:0.10810810810810811; {'Kleinfamilie'}:0.05405405405405406; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Basismaße für Schule=2: {{'Studierende'}:0.5675675675675675; set():0.33783783783783783; {'Kleinfamilie'}:0.0945945945945946; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}
Kombinierte Evidenz: {{'Kleinfamilie'}:1.0; {'Studierende'}:0.0; {'DINK'}:0.0; {'SingleHighIncome'}:0.0; {'Expatriate'}:0.0; {'Rentnerpaar'}:0.0}

Glaubwürdigkeit für alle Hypothesen:


AttributeError: 'MassFunction' object has no attribute 'get_hypotheses'